## Imports

In [1]:
import pandas as pd
import numpy as np
import configparser
import os
import sys

In [2]:
cwd = os.getcwd()

In [3]:
module_path = os.path.join(cwd, '..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import financialstatements

## Config

In [5]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\tasco\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Reading Data

### Bank Data

In [6]:
BANK_DIRECTORY = os.path.join(cwd,config.get("data_inputs_directory", "BANK_DIRECTORY"))

In [7]:
paths = financialstatements.creating_input_paths(BANK_DIRECTORY)

In [8]:
df = financialstatements.creating_df(paths)

## ETL

In [9]:
def processing_df(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Amount'] = df['Amount'].str.replace('$', '', regex=False)
    df['Amount'] = df['Amount'].str.replace(',', '')
    df['Amount'] = df['Amount'].astype('float')
    df['DEBIT'] = np.where(df['Amount'] < 0, df['Amount'], np.nan)
    df['DEBIT'] = df['DEBIT'].abs()
    df['CREDIT'] = np.where(df['Amount'] > 0, df['Amount'], np.nan)
    df['GL_Code'] = np.nan
    df['Account'] = np.nan
    df = df.sort_values(by='Date').reset_index(drop=True)
    df['Order_Col'] = df.index + 1
    df['Sub_Order_Col'] = np.where(df['DEBIT'].isnull(), 2, 1)
    return df[['Date', 'GL_Code', 'Account', 'Description','DEBIT', 'CREDIT', 'Order_Col', 'Sub_Order_Col']]

In [10]:
def removing_duplicates(df):
    keywords = ['WAL-MART ASSOCS. PAYROLL', 'CARDMEMBER SERV WEB PYMT']
    mask = df['Description'].str.contains('|'.join(keywords))
    return df[~mask]

In [11]:
def creating_credit_entries(df):
    df = df[df['CREDIT'].isnull()].reset_index(drop=True).copy()
    df['GL_Code'] = 100101
    df['Account'] = 'Free Checking Bank OZK'
    df['Sub_Order_Col'] = 2
    df = df.rename(columns={'DEBIT':'CREDIT', 'CREDIT': 'DEBIT'})
    return df[['Date', 'GL_Code', 'Account', 'Description','DEBIT','CREDIT','Order_Col', 'Sub_Order_Col']]

In [12]:
def creating_debit_entries(df):
    df = df[df['DEBIT'].isnull()].reset_index(drop=True).copy()
    df['GL_Code'] = 100101
    df['Account'] = 'Free Checking Bank OZK'
    df['Sub_Order_Col'] = 1
    df = df.rename(columns={'DEBIT':'CREDIT', 'CREDIT': 'DEBIT'})
    return df[['Date', 'GL_Code', 'Account', 'Description','DEBIT','CREDIT','Order_Col','Sub_Order_Col']]

In [13]:
df = processing_df(df)

In [14]:
df = removing_duplicates(df)

In [15]:
credit_entries_df = creating_credit_entries(df)

In [16]:
debit_entries_df = creating_debit_entries(df)

In [17]:
df = pd.concat([df, credit_entries_df, debit_entries_df])

In [18]:
df = df.sort_values(by=['Order_Col','Sub_Order_Col']).reset_index(drop=True)

In [19]:
type = 'bank'

In [20]:
financialstatements.creating_journal_entry_outputs(df, type, config, cwd)